In [14]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from typing import List
from tqdm import tqdm

from RandomSplit import RandomSplit
from metrics import ndcg_metric, dcg, recall_metric, evaluate_recommender, get_metrics

%matplotlib inline

import catalyst 
import recbole

from typing import Dict, List, Tuple

import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.nn.init import constant_, xavier_normal_
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset

from catalyst import dl, metrics
from catalyst.contrib.datasets import MovieLens
from catalyst.utils import get_device, set_global_seed
from torch.nn.utils.rnn import pad_sequence 

import random

from recbole.model.abstract_recommender import SequentialRecommender
from recbole.model.layers import *

from BERT4Rec import BERT4Rec
from runner import RecSysRunner
from ContextBERT4Rec import ContextBERT4Rec

from typing import Iterable, Any
from itertools import product

set_global_seed(100)
device = torch.device("mps")
print(device)

mps


### ML-100k

In [2]:
df = pd.read_csv('data/ML_100k.csv')
df = df.rename(columns={'userId': 'user_id', 'movieId': 'item_id'})
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df['weekday'] = pd.to_datetime(df.timestamp).dt.weekday
df['hour'] = pd.to_datetime(df.timestamp).dt.hour
df.head()

,user_id,item_id,rating,timestamp,weekday,hour
0,1,31,2.5,2009-12-14 02:52:24,0,2
1,1,1029,3.0,2009-12-14 02:52:59,0,2
2,1,1061,3.0,2009-12-14 02:53:02,0,2
3,1,1129,2.0,2009-12-14 02:53:05,0,2
4,1,1172,4.0,2009-12-14 02:53:25,0,2


In [3]:
splitter = RandomSplit(test_fraction=0.2)
train_df, valid_df, test_df = splitter(df)

In [4]:
train_grouped = train_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id, 
                                                                 x.timestamp,
                                                                 x.weekday,
                                                                 x.hour), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


train_grouped.head()

,user_id,train_interactions
0,1,"[(2294, 2009-12-14 02:51:48, 0, 2), (2455, 200..."
1,2,"[(150, 1996-06-21 11:09:55, 4, 11), (296, 1996..."
2,3,"[(355, 2011-02-28 02:53:09, 0, 2), (1271, 2011..."
3,4,"[(1210, 2000-02-05 19:25:14, 5, 19), (2734, 20..."
4,5,"[(1380, 2006-11-12 23:10:44, 6, 23), (1035, 20..."


In [5]:
joined = train_grouped.merge(valid_grouped).merge(test_grouped)
joined.head()

,user_id,train_interactions,valid_interactions,test_interactions
0,1,"[(2294, 2009-12-14 02:51:48, 0, 2), (2455, 200...","[(1287, 2009-12-14 02:53:07, 0, 2), (1953, 200...","[(2193, 2009-12-14 02:53:18, 0, 2), (2968, 200..."
1,2,"[(150, 1996-06-21 11:09:55, 4, 11), (296, 1996...","[(551, 1996-06-21 11:16:07, 4, 11), (273, 1996...","[(319, 1996-06-21 11:18:38, 4, 11), (485, 1996..."
2,3,"[(355, 2011-02-28 02:53:09, 0, 2), (1271, 2011...","[(2028, 2011-02-28 19:37:42, 0, 19), (110, 201...","[(1721, 2011-02-28 20:00:36, 0, 20), (377, 201..."
3,4,"[(1210, 2000-02-05 19:25:14, 5, 19), (2734, 20...","[(3169, 2000-02-07 10:21:44, 0, 10), (1298, 20...","[(1858, 2000-02-07 10:35:38, 0, 10), (3108, 20..."
4,5,"[(1380, 2006-11-12 23:10:44, 6, 23), (1035, 20...","[(1968, 2006-11-12 23:32:04, 6, 23), (30707, 2...","[(33679, 2006-11-12 23:35:17, 6, 23), (5995, 2..."


In [6]:
our_items = set()
for idx, row in tqdm(joined.iterrows()):
    for el in row.train_interactions:
        our_items.add(el[0])
        
len(our_items)

671it [00:00, 29267.05it/s]


6811

In [7]:
item2idx = {k: i for i, k in enumerate(our_items)}
idx2item = {i: k for k, i in item2idx.items()}

In [ ]:
class MyDataset(Dataset):
    
    def __init__(self, ds, num_items, item2idx, phase='valid', N=200):
        super().__init__()
        self.ds = ds
        self.phase = phase
        self.n_items = num_items
        self.item2idx = item2idx
        self.N = N 
        
    def __len__(self):
        return len(self.ds)
    
    def __getitem__(self, idx):
        
        row = self.ds.iloc[idx]
        
        x_input = np.zeros(self.n_items+1)
        x_input[[self.item2idx[x[0]]+1 for x in row['train_interactions'] if x[0] in self.item2idx]] = 1
        
        days_of_weeks = [x[2] for x in row['train_interactions'] if x[0] in self.item2idx][-self.N+1:]
        
        hours = [x[3] for x in row['train_interactions'] if x[0] in self.item2idx][-self.N+1:]
        
        seq_input = [self.item2idx[x[0]]+1 for x in row['train_interactions'] if x[0] in self.item2idx][-self.N+1:]
        
        targets = np.zeros(self.n_items+1)
        
        dow_valid = row['valid_interactions'][0][2]
        dow_test = row['test_interactions'][0][2]
        
        hours_valid = row['valid_interactions'][0][3]
        hours_test = row['test_interactions'][0][3]
        
        if self.phase == 'train':
            return (seq_input, days_of_weeks, hours, dow_valid, hours_valid)
        elif self.phase == 'valid':
            targets[[self.item2idx[x[0]]+1 for x in row['valid_interactions'] if x[0] in self.item2idx]] = 1
        else:
            return (seq_input, days_of_weeks, hours, dow_test, hours_test)
            
        return (targets, seq_input, days_of_weeks, hours, dow_valid, hours_valid)
     
n_items = len(item2idx)

train = MyDataset(ds=joined,
                  num_items=n_items, 
                  item2idx=item2idx,
                  phase='train')

valid = MyDataset(ds=joined,
                  num_items=n_items,
                  item2idx=item2idx,
                  phase='valid')

print(len(train),len(valid))

671 671


In [9]:
def collate_fn_train(batch: List[Tuple[torch.Tensor]]) -> Dict[str, torch.Tensor]: 
    
    seq_i,days_of_weeks,hours,dow_valid,hours_valid = zip(*batch)
    seq_len = torch.Tensor([len(x) for x in seq_i])
    dow_valid = torch.Tensor([x for x in dow_valid])
    hours_valid = torch.Tensor([x for x in hours_valid])
    seq_i = pad_sequence([torch.Tensor(t) for t in seq_i]).T    
    days_of_weeks = pad_sequence([torch.Tensor(t) for t in days_of_weeks]).T
    hours = pad_sequence([torch.Tensor(t) for t in hours]).T
    
    return {'seq_i': seq_i, 
            'seq_len':seq_len,
            'dow': days_of_weeks,
            'hours': hours,
            'dow_valid': dow_valid,
            'hours_valid': hours_valid}


def collate_fn_valid(batch: List[Tuple[torch.Tensor]]) -> Dict[str, torch.Tensor]:
    
    y, seq_i, days_of_weeks, hours, dow_valid, hours_valid = zip(*batch)
    
    seq_len = torch.Tensor([len(x) for x in seq_i]).long()
    seq_i = pad_sequence([torch.Tensor(t) for t in seq_i]).T.long()
    days_of_weeks = pad_sequence([torch.Tensor(t) for t in days_of_weeks]).T.long()
    hours = pad_sequence([torch.Tensor(t) for t in hours]).T.long()
    dow_valid = torch.Tensor([x for x in dow_valid])
    hours_valid = torch.Tensor([x for x in hours_valid])
            
    targets = pad_sequence([torch.Tensor(t) for t in y]).T

    return {"targets": targets,
            'seq_i': seq_i,
            'seq_len':seq_len,
            'dow': days_of_weeks,
            'hours': hours,
            'dow_valid': dow_valid,
            'hours_valid': hours_valid}

In [10]:
loaders = {
        "train": DataLoader(train, batch_size=256, collate_fn=collate_fn_train),
        "valid": DataLoader(valid, batch_size=256, collate_fn=collate_fn_valid),
}

In [17]:
def grid_parameters(parameters: dict[str, Iterable[Any]]) -> Iterable[dict[str, Any]]:
    for params in product(*parameters.values()):
        yield dict(zip(parameters.keys(), params))

In [18]:
param_grid = {
    'lr': [0.01, 0.001, 0.0001],
    'mask_ratio': [0.15, 0.2, 0.4],
    'dropout': [0.1, 0.2, 0.9],
    'dimensionality': [32, 64, 128]
}

In [23]:
results = pd.DataFrame(columns = ['lr', 'mask_ratio', 'dropout', 'dimensionality', 'map10', 'ndcg']) 

for settings in grid_parameters(param_grid):


    model = ContextBERT4Rec(n_items=len(item2idx)+1, mask_ratio=settings['mask_ratio'], hidden_size=settings['dimensionality']
                           ,hidden_dropout_prob=settings['dropout'])

    optimizer = optim.Adam(model.parameters(), lr=settings['lr'])
    lr_scheduler = StepLR(optimizer, step_size=20, gamma=0.1)
    engine = dl.DeviceEngine('cpu')
    hparams = {
        "anneal_cap": 0.2,
        "total_anneal_steps": 6000,
    }


    callbacks = [
        dl.NDCGCallback("logits", "targets", [20]),
        dl.MAPCallback("logits", "targets", [10]),
        dl.OptimizerCallback("loss", accumulation_steps=1),
        dl.EarlyStoppingCallback(
            patience=5, loader_key="valid", metric_key="map10", minimize=False
        )
    ]


    runner = RecSysRunner()
    runner.train(
        model=model,
        optimizer=optimizer,
        engine=engine,
        hparams=hparams,
        scheduler=lr_scheduler,
        loaders=loaders,
        num_epochs=100,
        verbose=True,
        timeit=True,
        callbacks=callbacks,
    )
    
    metrics = runner.evaluate_loader(loader=loaders["valid"],
                                    callbacks=[dl.MAPCallback("logits", "targets", [10]), dl.NDCGCallback("logits", "targets", [20])],
                )
    
    
    results=results.append({'lr': settings['lr'], 
                            'mask_ratio': settings['mask_ratio'],
                           'dropout': settings['dropout'],
                           'dimensionality': settings['dimensionality'],
                           'map10': metrics['map10'],
                           'ndcg': metrics['ndcg20']}, ignore_index=True)

1/100 * Epoch (train): 100%|██████████| 3/3 [00:02<00:00,  1.42it/s, _timer/_fps=260.080, _timer/batch_time=0.611, _timer/data_time=0.333, _timer/model_time=0.279, loss=8.530, lr=0.010, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]


train (1/100) loss: 8.750721581824312 | lr: 0.01 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


1/100 * Epoch (valid): 100%|██████████| 3/3 [00:01<00:00,  2.36it/s, _timer/_fps=586.352, _timer/batch_time=0.271, _timer/data_time=0.023, _timer/model_time=0.249, loss=8.281, lr=0.010, map10=0.096, momentum=0.900, ndcg20=0.048]


valid (1/100) loss: 8.2187116696831 | lr: 0.01 | map10: 0.08383750305934386 | map10/std: 0.0072205957642250814 | momentum: 0.9 | ndcg20: 0.041338570660254995 | ndcg20/std: 0.003566892319778399
* Epoch (1/100) 


2/100 * Epoch (train): 100%|██████████| 3/3 [00:02<00:00,  1.41it/s, _timer/_fps=270.564, _timer/batch_time=0.588, _timer/data_time=0.334, _timer/model_time=0.254, loss=8.142, lr=0.010, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]


...

47/100 * Epoch (train): 100%|██████████| 3/3 [00:05<00:00,  1.70s/it, _timer/_fps=102.754, _timer/batch_time=1.547, _timer/data_time=0.987, _timer/model_time=0.560, loss=8.065, lr=1.000e-04, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]


train (47/100) loss: 8.065966142805074 | lr: 0.0001 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


47/100 * Epoch (valid): 100%|██████████| 3/3 [00:02<00:00,  1.28it/s, _timer/_fps=292.473, _timer/batch_time=0.544, _timer/data_time=0.026, _timer/model_time=0.517, loss=7.803, lr=1.000e-04, map10=0.167, momentum=0.900, ndcg20=0.086]


valid (47/100) loss: 7.79350122502948 | lr: 0.0001 | map10: 0.1285997148597649 | map10/std: 0.021163906866575134 | momentum: 0.9 | ndcg20: 0.07382294504546551 | ndcg20/std: 0.0066542116799870295
* Epoch (47/100) 
valid (1/1) loss: 7.7988983358960215 | map10: 0.1285997148597649 | map10/std: 0.021163906866575134 | ndcg20: 0.07382294504546551 | ndcg20/std: 0.0066542116799870295
* Epoch (1/1) 


/var/folders/l1/qss5xzq52gv8x0_f8b47flw00000gn/T/ipykernel_4627/568892883.py:47: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results=results.append({'lr': settings['lr'],


In [24]:
results

,lr,mask_ratio,dropout,dimensionality,map10,ndcg
0,0.0100,0.15,0.1,32.0,0.122646,0.072598
1,0.0100,0.15,0.1,64.0,0.132739,0.071733
2,0.0100,0.15,0.1,128.0,0.122263,0.071576
3,0.0100,0.15,0.2,32.0,0.124768,0.074576
4,0.0100,0.15,0.2,64.0,0.134617,0.075644
...,...,...,...,...,...,...
76,0.0001,0.40,0.2,64.0,0.124422,0.067316
77,0.0001,0.40,0.2,128.0,0.120438,0.073338
78,0.0001,0.40,0.9,32.0,0.117042,0.067174
79,0.0001,0.40,0.9,64.0,0.110887,0.060383


In [40]:
results.iloc[results['map10'].idxmax()]

lr                  0.010000
mask_ratio          0.200000
dropout             0.200000
dimensionality    128.000000
map10               0.140728
ndcg                0.080409
Name: 14, dtype: float64

### ML-1m

In [25]:
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
df = pd.read_table('data/ratings.dat', sep='::',header=None, names=rnames, engine='python')
df = df.rename(columns={'userId': 'user_id', 'movie_id': 'item_id'})
df['timestamp'] = pd.to_datetime(df['timestamp'],unit='s')
df['weekday'] = pd.to_datetime(df.timestamp).dt.weekday
df['hour'] = pd.to_datetime(df.timestamp).dt.hour
df.head()

,user_id,item_id,rating,timestamp,weekday,hour
0,1,1193,5,2000-12-31 22:12:40,6,22
1,1,661,3,2000-12-31 22:35:09,6,22
2,1,914,3,2000-12-31 22:32:48,6,22
3,1,3408,4,2000-12-31 22:04:35,6,22
4,1,2355,5,2001-01-06 23:38:11,5,23


In [26]:
splitter = RandomSplit(test_fraction=0.2)
train_df, valid_df, test_df = splitter(df)

In [27]:
train_grouped = train_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id, 
                                                                 x.timestamp,
                                                                 x.weekday,
                                                                 x.hour), key=lambda x: x[1])]
).reset_index()
train_grouped.rename({0:'train_interactions'}, axis=1, inplace=True)

valid_grouped = valid_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
valid_grouped.rename({0:'valid_interactions'}, axis=1, inplace=True)

test_grouped = test_df.groupby('user_id').apply(
    lambda x: [(t1, t2, t3, t4) for t1, t2, t3, t4 in sorted(zip(x.item_id,
                                                         x.timestamp,
                                                         x.weekday,
                                                         x.hour), key=lambda x: x[1])]
).reset_index()
test_grouped.rename({0:'test_interactions'}, axis=1, inplace=True)


train_grouped.head()

,user_id,train_interactions
0,1,"[(3186, 2000-12-31 22:00:19, 6, 22), (1270, 20..."
1,2,"[(1198, 2000-12-31 21:28:44, 6, 21), (1210, 20..."
2,3,"[(593, 2000-12-31 21:10:18, 6, 21), (2858, 200..."
3,4,"[(1210, 2000-12-31 20:18:44, 6, 20), (1097, 20..."
4,5,"[(2717, 2000-12-31 05:37:52, 6, 5), (908, 2000..."


In [28]:
joined = train_grouped.merge(valid_grouped).merge(test_grouped)
joined.head()

,user_id,train_interactions,valid_interactions,test_interactions
0,1,"[(3186, 2000-12-31 22:00:19, 6, 22), (1270, 20...","[(2791, 2000-12-31 22:36:28, 6, 22), (2321, 20...","[(2687, 2001-01-06 23:37:48, 5, 23), (745, 200..."
1,2,"[(1198, 2000-12-31 21:28:44, 6, 21), (1210, 20...","[(2028, 2000-12-31 21:56:13, 6, 21), (2571, 20...","[(1372, 2000-12-31 21:59:01, 6, 21), (1552, 20..."
2,3,"[(593, 2000-12-31 21:10:18, 6, 21), (2858, 200...","[(648, 2000-12-31 21:24:27, 6, 21), (2735, 200...","[(1270, 2000-12-31 21:30:31, 6, 21), (1079, 20..."
3,4,"[(1210, 2000-12-31 20:18:44, 6, 20), (1097, 20...","[(2947, 2000-12-31 20:23:50, 6, 20), (1214, 20...","[(1240, 2000-12-31 20:24:20, 6, 20), (2951, 20..."
4,5,"[(2717, 2000-12-31 05:37:52, 6, 5), (908, 2000...","[(2323, 2000-12-31 06:50:45, 6, 6), (272, 2000...","[(1715, 2000-12-31 06:58:11, 6, 6), (1653, 200..."


In [29]:
our_items = set()
for idx, row in tqdm(joined.iterrows()):
    for el in row.train_interactions:
        our_items.add(el[0])
        
len(our_items)

6040it [00:00, 35473.08it/s]


3636

In [30]:
item2idx = {k: i for i, k in enumerate(our_items)}
idx2item = {i: k for k, i in item2idx.items()}

In [31]:
n_items = len(item2idx)

train = MyDataset(ds=joined,
                  num_items=n_items, 
                  item2idx=item2idx,
                  phase='train')

valid = MyDataset(ds=joined,
                  num_items=n_items,
                  item2idx=item2idx,
                  phase='valid')

print(len(train),len(valid))

6040 6040


In [32]:
loaders = {
        "train": DataLoader(train, batch_size=256, collate_fn=collate_fn_train),
        "valid": DataLoader(valid, batch_size=256, collate_fn=collate_fn_valid),
}

In [33]:
results2 = pd.DataFrame(columns = ['lr', 'mask_ratio', 'dropout', 'dimensionality', 'map10', 'ndcg']) 

for settings in grid_parameters(param_grid):


    model = ContextBERT4Rec(n_items=len(item2idx)+1, mask_ratio=settings['mask_ratio'], hidden_size=settings['dimensionality']
                           ,hidden_dropout_prob=settings['dropout'])

    optimizer = optim.Adam(model.parameters(), lr=settings['lr'])
    lr_scheduler = StepLR(optimizer, step_size=20, gamma=0.1)
    engine = dl.DeviceEngine('cpu')
    hparams = {
        "anneal_cap": 0.2,
        "total_anneal_steps": 6000,
    }


    callbacks = [
        dl.NDCGCallback("logits", "targets", [20]),
        dl.MAPCallback("logits", "targets", [10]),
        dl.OptimizerCallback("loss", accumulation_steps=1),
        dl.EarlyStoppingCallback(
            patience=5, loader_key="valid", metric_key="map10", minimize=False
        )
    ]


    runner = RecSysRunner()
    runner.train(
        model=model,
        optimizer=optimizer,
        engine=engine,
        hparams=hparams,
        scheduler=lr_scheduler,
        loaders=loaders,
        num_epochs=100,
        verbose=True,
        timeit=True,
        callbacks=callbacks,
    )
    
    metrics = runner.evaluate_loader(loader=loaders["valid"],
                                    callbacks=[dl.MAPCallback("logits", "targets", [10]), dl.NDCGCallback("logits", "targets", [20])],
                )
    
    
    results2=results2.append({'lr': settings['lr'], 
                            'mask_ratio': settings['mask_ratio'],
                           'dropout': settings['dropout'],
                           'dimensionality': settings['dimensionality'],
                           'map10': metrics['map10'],
                           'ndcg': metrics['ndcg20']}, ignore_index=True)

1/100 * Epoch (train): 100%|██████████| 24/24 [00:16<00:00,  1.45it/s, _timer/_fps=304.862, _timer/batch_time=0.499, _timer/data_time=0.231, _timer/model_time=0.267, loss=7.380, lr=0.010, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]


train (1/100) loss: 7.574837868418915 | lr: 0.01 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


1/100 * Epoch (valid): 100%|██████████| 24/24 [00:10<00:00,  2.27it/s, _timer/_fps=482.886, _timer/batch_time=0.315, _timer/data_time=0.028, _timer/model_time=0.287, loss=7.336, lr=0.010, map10=0.110, momentum=0.900, ndcg20=0.063]


valid (1/100) loss: 7.384134677230127 | lr: 0.01 | map10: 0.13321730025754067 | map10/std: 0.019330944708003617 | momentum: 0.9 | ndcg20: 0.07583214861667709 | ndcg20/std: 0.008654961337384092
* Epoch (1/100) 


2/100 * Epoch (train): 100%|██████████| 24/24 [00:16<00:00,  1.44it/s, _timer/_fps=286.781, _timer/batch_time=0.530, _timer/data_time=0.254, _timer/model_time=0.276, loss=7.343, lr=0.010, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]


...

7/100 * Epoch (train): 100%|██████████| 24/24 [00:35<00:00,  1.50s/it, _timer/_fps=118.985, _timer/batch_time=1.277, _timer/data_time=0.748, _timer/model_time=0.530, loss=7.344, lr=0.010, map10=0.000e+00, momentum=0.900, ndcg20=0.000e+00]


train (7/100) loss: 7.382406454528405 | lr: 0.01 | map10: 0.0 | map10/std: 0.0 | momentum: 0.9 | ndcg20: 0.0 | ndcg20/std: 0.0


7/100 * Epoch (valid):  71%|███████   | 17/24 [00:13<00:05,  1.23it/s, _timer/_fps=356.210, _timer/batch_time=0.719, _timer/data_time=0.041, _timer/model_time=0.678, loss=7.467, lr=0.010, map10=0.104, momentum=0.900, ndcg20=0.062]

Keyboard Interrupt


KeyboardInterrupt: 

In [35]:
results2.head()

,lr,mask_ratio,dropout,dimensionality,map10,ndcg
0,0.01,0.15,0.1,32.0,0.136576,0.078137
1,0.01,0.15,0.1,64.0,0.139044,0.079418
2,0.01,0.15,0.1,128.0,0.136785,0.079578
3,0.01,0.15,0.2,32.0,0.142781,0.078298
4,0.01,0.15,0.2,64.0,0.134090,0.079905


In [41]:
results2.iloc[results2['map10'].idxmax()]

lr                  0.010000
mask_ratio          0.200000
dropout             0.900000
dimensionality    128.000000
map10               0.143022
ndcg                0.080778
Name: 17, dtype: float64